In [3]:
import optuna
import numpy as np
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, load_metric

# Create a larger synthetic dataset with more varied examples
data = [
    {"tokens": ["Le", "patient", "a", "de", "la", "fièvre", "et", "des", "maux", "de", "tête", "."],
     "ner_tags": [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]},
    {"tokens": ["Il", "souffre", "de", "douleurs", "abdominales", "et", "de", "nausées", "."],
     "ner_tags": [0, 1, 0, 1, 1, 0, 0, 1, 0]},
    {"tokens": ["La", "patiente", "présente", "une", "toux", "et", "des", "difficultés", "respiratoires", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Elle", "a", "des", "douleurs", "thoraciques", "et", "une", "fièvre", "élevée", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Le", "patient", "a", "un", "rhume", "et", "se", "plaind", "de", "maux", "de", "gorge", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0]},
    {"tokens": ["Elle", "est", "fatiguée", "et", "a", "des", "douleurs", "musculaires", "."],
     "ner_tags": [0, 0, 1, 0, 0, 0, 1, 1, 0]},
    {"tokens": ["Il", "présente", "des", "éruptions", "cutanées", "et", "de", "la", "fièvre", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Le", "patient", "a", "des", "vertiges", "et", "des", "troubles", "de", "la", "vision", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]},
    {"tokens": ["Elle", "souffre", "de", "migraine", "et", "a", "des", "nausées", "."],
     "ner_tags": [0, 1, 0, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Il", "a", "une", "grosse", "fièvre", "et", "se", "plaind", "de", "frissons", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0]},
    
    # Additional varied examples
    {"tokens": ["Le", "malade", "a", "des", "courbatures", "et", "de", "la", "fatigue", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Elle", "ressent", "une", "douleur", "au", "ventre", "et", "a", "perdu", "l'appétit", "."],
     "ner_tags": [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Il", "a", "des", "démangeaisons", "et", "une", "éruption", "cutanée", "."],
     "ner_tags": [0, 0, 0, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Le", "patient", "se", "plaint", "de", "douleurs", "articulaires", "et", "de", "fatigue", "."],
     "ner_tags": [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0]},
    {"tokens": ["Elle", "a", "des", "difficultés", "à", "respirer", "et", "de", "la", "fièvre", "."],
     "ner_tags": [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Il", "a", "perdu", "du", "poids", "et", "se", "sent", "faible", "."],
     "ner_tags": [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]},
    {"tokens": ["Le", "patient", "a", "une", "sensation", "de", "brûlure", "à", "la", "miction", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]},
    {"tokens": ["Elle", "a", "des", "problèmes", "digestifs", "et", "se", "sent", "ballonnée", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Il", "se", "plaint", "de", "douleurs", "au", "dos", "et", "de", "raideurs", "musculaires", "."],
     "ner_tags": [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Elle", "a", "des", "douleurs", "menstruelles", "et", "des", "nausées", "pendant", "ses", "règles", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0]}
]

# Convert to Dataset
dataset = Dataset.from_list(data)

# Load tokenizer
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")

# Define label names
label_names = ["O", "B-ILLNESS"]

# Function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True, padding='max_length', max_length=128)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization and alignment
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Split into train and validation sets
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

# Load metric
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Hyperparameter search function
def objective(trial):
    # Define hyperparameters to search
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    num_train_epochs = trial.suggest_int('num_train_epochs', 2, 10)
    per_device_train_batch_size = trial.suggest_categorical('per_device_train_batch_size', [4, 8, 16])

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        logging_dir='./logs',
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    return eval_results['eval_f1']

# Load model
model = CamembertForTokenClassification.from_pretrained("camembert-base", num_labels=len(label_names))

# Hyperparameter search with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Print best hyperparameters
print(study.best_params)

# Train final model with best hyperparameters
best_params = study.best_params
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=best_params['learning_rate'],
    per_device_train_batch_size=best_params['per_device_train_batch_size'],
    per_device_eval_batch_size=best_params['per_device_train_batch_size'],
    num_train_epochs=best_params['num_train_epochs'],
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the final model
trainer.train()


# Save the final model
output_dir = "./saved_model"  # Directory where you want to save the model
trainer.save_model(output_dir)


# Evaluate the final model
final_results = trainer.evaluate()
print(final_results)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

C:\Users\User\AppData\Local\Temp\ipykernel_11756\2366498727.py:90: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
C:\Users\User\anaconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier

[I 2024-06-13 08:29:54,302] Trial 0 finished with value: 0.9230769230769231 and parameters: {'learning_rate': 2.106232819184001e-05, 'num_train_epochs': 9, 'per_device_train_batch_size': 4}. Best is trial 0 with value: 0.9230769230769231.


[I 2024-06-13 08:30:42,342] Trial 1 finished with value: 0.888888888888889 and parameters: {'learning_rate': 1.2740897698033196e-05, 'num_train_epochs': 7, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.9230769230769231.


[I 2024-06-13 08:31:32,161] Trial 2 finished with value: 0.888888888888889 and parameters: {'learning_rate': 1.968558886508463e-05, 'num_train_epochs': 7, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.9230769230769231.


...

[I 2024-06-13 08:36:13,843] Trial 7 finished with value: 0.888888888888889 and parameters: {'learning_rate': 1.5392777921201707e-05, 'num_train_epochs': 7, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.9230769230769231.


[I 2024-06-13 08:36:56,485] Trial 8 finished with value: 0.888888888888889 and parameters: {'learning_rate': 1.3118957676773008e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 4}. Best is trial 0 with value: 0.9230769230769231.


[I 2024-06-13 08:37:30,654] Trial 9 finished with value: 0.888888888888889 and parameters: {'learning_rate': 3.924335112281607e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.9230769230769231.


{'learning_rate': 2.106232819184001e-05, 'num_train_epochs': 9, 'per_device_train_batch_size': 4}


{'eval_loss': 0.2625805735588074, 'eval_precision': 0.8571428571428571, 'eval_recall': 0.9230769230769231, 'eval_f1': 0.888888888888889, 'eval_accuracy': 0.9302325581395349, 'eval_runtime': 0.4999, 'eval_samples_per_second': 8.002, 'eval_steps_per_second': 2.0, 'epoch': 9.0}
